# Testing Notebook for Keysight DSOX3024A Oscilloscope Driver

This notebook is designed to walk a technician through testing the functionality of the `KeysightDSOX3024a` driver. 

**Instructions for the Technician:**
1. Ensure a Keysight DSOX3024A oscilloscope is connected to the same network as this computer.
2. Run each cell sequentially.
3. After running a cell that sends a command to the instrument, observe the oscilloscope's screen to verify that the expected change has occurred.
4. For data acquisition cells, check the output in the notebook to ensure it is reasonable.

## 1. Connect to the Instrument

In [1]:
from piec.drivers.Keysight.k_dsox3024a import KeysightDSOX3024a
from piec.drivers.utilities import PiecManager

# Initialize PiecManager and list available resources
pm = PiecManager()
pm.list_resources()

('GPIB0::7::INSTR', 'GPIB0::8::INSTR')

**Technician Note:** Find the VISA address for the Keysight oscilloscope in the list above and replace `'Address'` with the correct string (e.g., `'TCPIP0::192.168.1.1::inst0::INSTR'`).

In [2]:
# Connect to the instrument
# The technician will replace 'Address' with the actual instrument address from the list above.
scope = KeysightDSOX3024a("GPIB0::7::INSTR")  # Replace with actual address

## 2. Basic Functionality Test

In [28]:
# Test Autoscale
scope.autoscale()

**Expected Output:** The oscilloscope should perform an autoscale operation, adjusting the vertical and horizontal scales to fit any connected signal on the active channels.

## 3. Channel Configuration Tests

In [4]:
# Turn Channel 2 Off
scope.toggle_channel(channel=2, on=False)

**Expected Output:** The waveform and display for Channel 2 should disappear from the oscilloscope screen.

In [5]:
# Turn Channel 2 On
scope.toggle_channel(channel=2, on=True)

**Expected Output:** Channel 2 should reappear on the oscilloscope screen.

In [6]:
# Set Vertical Scale for Channel 1 (Volts/Division)
scope.set_vertical_scale(channel=1, vdiv=0.5)

**Expected Output:** The vertical scale for Channel 1 should be set to 500 mV/div.

In [7]:
# Set Vertical Scale for Channel 1 (Absolute Range)
scope.set_vertical_scale(channel=1, y_range=10.0)

**Expected Output:** The vertical range for Channel 1 should be set to 10.0 V. This corresponds to 1.25 V/div (10V / 8 divisions).

In [8]:
# Set Vertical Position for Channel 1
scope.set_vertical_position(channel=1, y_position=-1.0)

**Expected Output:** The waveform for Channel 1 should shift downwards on the screen. The ground level (0V) for Channel 1 will be shifted up by one division (assuming 1 V/div).

In [11]:
# Set Input Coupling for Channel 1 to AC
scope.set_input_coupling(channel=1, input_coupling='AC')

**Expected Output:** The input coupling for Channel 1 should be set to AC. This will be indicated on the oscilloscope's display for that channel.

In [14]:
# Set Probe Attenuation for Channel 1 to 10x
scope.set_probe_attenuation(channel=1, probe_attenuation=10)

**Expected Output:** The probe attenuation for Channel 1 should be set to 10x. All voltage readings and scales for Channel 1 will be multiplied by 10.

## 4. Horizontal Configuration Tests

In [15]:
# Set Horizontal Scale (Time/Division)
scope.set_horizontal_scale(tdiv=0.001)

**Expected Output:** The horizontal scale should be set to 1 ms/div.

In [16]:
# Set Horizontal Position (Delay)
scope.set_horizontal_position(x_position=0.0005)

**Expected Output:** The trigger point should shift to the left on the screen, introducing a 500 µs delay.

In [17]:
# Configure Horizontal settings all at once
scope.configure_horizontal(tdiv=0.0001, x_position=0)

**Expected Output:** The horizontal scale should be set to 100 µs/div and the horizontal position (delay) should be reset to 0.

## 5. Trigger Configuration Tests

In [ ]:
# Configure all trigger settings at once
scope.configure_trigger(trigger_source='CHAN1', trigger_level=1.5, trigger_slope='POS', trigger_mode='EDGE')

**Expected Output:** The oscilloscope trigger should be configured as follows:
- **Source:** Channel 1
- **Mode:** Edge
- **Level:** 1.5 V
- **Slope:** Positive (Rising)

In [20]:
# Set the trigger sweep to Normal
scope.set_trigger_sweep(trigger_sweep='NORM')

**Expected Output:** The oscilloscope's trigger sweep mode should be set to 'Normal'. The scope will only trigger and update the display when a valid trigger event occurs.

## 6. Acquisition and Data Retrieval Tests

In [21]:
# Stop Acquisition
scope.toggle_acquisition(run=False)

**Expected Output:** The oscilloscope should stop acquiring data. The 'Run/Stop' button on the front panel should indicate 'Stop' (typically red).

In [32]:
# Configure the acquisition parameters
scope.configure_acquisition(channel=1, acquisition_mode='NORM', acquisition_points=10000)

**Expected Output:** The oscilloscope's acquisition system should be configured for:
- **Waveform Source:** Channel 1
- **Acquisition Mode:** High Resolution
- **Number of Points:** 10,000

In [33]:
# Arm the scope for a single shot
scope.arm()

**Expected Output:** The oscilloscope should be armed for a single acquisition. The 'Single' button on the front panel should be illuminated. The scope will now wait for a trigger event.

**Technician Note:** At this point, ensure a signal that meets the trigger conditions (Channel 1, rising edge crosses 1.5V) is present to capture a waveform.

In [34]:
# Perform a quick read of the data on screen
# Note: This uses different settings than we just configured
quick_data = scope.quick_read()
print(f"Quick read successful. Acquired {quick_data.shape[0]} points.")
print(quick_data)

Quick read successful. Acquired 8000 points.
[128 128 128 ... 128 128 128]


**Expected Output:** The notebook should print a success message and a numpy array of integer values. This data represents the raw byte values of the waveform currently visible on the screen.

In [35]:
# Get the fully configured data
# This uses the settings from the 'configure_acquisition' call
data_df = scope.get_data()
print("Configured data read successful.")
data_df.head()

Configured data read successful.


,Time,Voltage
0,-1.000000e-06,-51.317288
1,-9.997500e-07,-51.317288
2,-9.995000e-07,-51.317288
3,-9.992500e-07,-51.116283
4,-9.990000e-07,-51.116283


**Expected Output:** The notebook should print a success message and display the first 5 rows of a Pandas DataFrame. The DataFrame should have two columns: 'Time' and 'Voltage', containing the calculated time and voltage values for the 10,000 acquired points.

In [36]:
# Start Acquisition again
scope.toggle_acquisition(run=True)

**Expected Output:** The oscilloscope should resume continuous acquisition. The 'Run/Stop' button should indicate 'Run' (typically green).

## 7. End of Test

If all the steps above were completed and the observed behavior on the oscilloscope matched the expected output, the driver is functioning correctly.

In [37]:
scope.reset()